In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp



In [12]:
import tensorflow as tf
from keras import optimizers
from keras import callbacks
from keras.datasets import cifar10
from keras.engine import Model
from keras.applications import vgg16 as vgg
from keras.layers import Dropout,Flatten,Dense,GlobalAveragePooling2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

In [6]:
BATCH_SIZE=32
EPOCHS=40
NUM_CLASSES=10
LEARNING_RATE =1e-4
MOMENTUM=0.9

In [8]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.15,stratify = np.array(y_train),random_state=0)

In [19]:
y_train = np_utils.to_categorical(y_train,NUM_CLASSES)
y_test = np_utils.to_categorical(y_test,NUM_CLASSES)
y_val = np_utils.to_categorical(y_val,NUM_CLASSES)

In [25]:
x_train = np.array([sp.misc.imresize(x,(48,48)) for x in x_train])
x_test = np.array([sp.misc.imresize(x,(48,48)) for x in x_test])
x_val = np.array([sp.misc.imresize(x,(48,48)) for x in x_val])

C:\Users\saipr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """Entry point for launching an IPython kernel.
C:\Users\saipr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  
C:\Users\saipr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
base_model = vgg.VGG16(weights='imagenet',include_top=False,input_shape =(48,48,3))
base_model

In [30]:
last = base_model.get_layer('block3_pool').output

In [34]:
x = GlobalAveragePooling2D()(last)
x = BatchNormalization()(x)
x = Dense(256,activation='relu')(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.6)(x)
pred = Dense(NUM_CLASSES,activation='softmax')(x)
model = Model(base_model.input,pred)


In [35]:
for layer in base_model.layers:
    layer.trainable = False

In [36]:
model.compile(loss ='binary_crossentropy',optimizer = optimizers.adam(lr=LEARNING_RATE),metrics=['accuracy'])

In [38]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________